In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.0` // Or use any other 2.x version here
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import $ivy.$                                   // Or use any other 2.x version here

import org.apache.log4j.{Level, Logger}


In [2]:
import org.apache.spark.SparkContext

import org.apache.spark.SparkContext

In [5]:
val sc= new SparkContext("local[*]", "inverted index")

sc: SparkContext = org.apache.spark.SparkContext@5ba95b83

In [6]:
val separator= java.io.File.separator

separator: String = "/"

In [27]:
val inpath= "data/enron-spam-ham/*"

inpath: String = "data/enron-spam-ham/*"

In [28]:
val outpath= "output/crawl"

outpath: String = "output/crawl"

In [29]:
val files_contents= sc.wholeTextFiles(inpath).map{
    case (id,text) =>
        val id2= id.split("/").last
        (id2,text.trim.replaceAll("""([\p{Punct}&&[^.]]|\b\p{IsLetter}{1,2}\b)\s*|\s*\n\s*""", " "))
}

files_contents: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[34] at map at cmd28.sc:1

In [31]:
files_contents.saveAsTextFile(outpath)

In [32]:
files_contents.take(1)

res31: Array[(String, String)] = Array(
  ("0001.1999-12-10.farmer.ham.txt", "Subject christmas tree farm pictures")
)

In [33]:
val outpath2= "output/inverted-index"

outpath2: String = "output/inverted-index"

In [34]:
val input= sc.textFile(outpath)

input: org.apache.spark.rdd.RDD[String] = output/crawl MapPartitionsRDD[38] at textFile at cmd33.sc:1

In [35]:
val index= files_contents.flatMap{
    case (path,text)=>
        text.trim.split("""[^\w]""").map(word=> ((word,path),1))
}
.reduceByKey(_+_)
.map{case ((word,path),n) => (word,(path,n))}
.groupByKey
.map {case(word, iterable)=> 
    val vec= iterable.to[Vector].sortBy{case (path,n) => (-n,path)}
    (word,vec.mkString(", "))
}


index: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[43] at map at cmd34.sc:8

In [36]:
index.take(5)

res35: Array[(String, String)] = Array(
  ("sellens", "(0009.2000-06-07.lokay.ham.txt,1)"),
  ("pitifully", "(0013.2004-08-01.BG.spam.txt,1)"),
  (
    "someone",
    "(0027.1999-12-16.kaminski.ham.txt,5), (0016.2001-07-06.SA_and_HP.spam.txt,4), (0019.2001-07-06.SA_and_HP.spam.txt,4), (0011.2001-06-29.SA_and_HP.spam.txt,3), (0013.2001-06-30.SA_and_HP.spam.txt,3), (0002.1999-12-13.farmer.ham.txt,1), (0029.2001-08-03.SA_and_HP.spam.txt,1), (0030.1999-12-20.farmer.ham.txt,1)"
  ),
  ("bone", "(0006.2003-12-18.GP.spam.txt,1)"),
  ("darlington", "(0023.2004-08-02.BG.spam.txt,1)")
)

In [37]:
index.saveAsTextFile(outpath2)

In [38]:
sc.stop()